In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [2]:
import os
import joblib
data_path = '/home/pavel/P/kaggle_data/talking_data'

In [3]:
app_events = pd.read_csv(os.path.join(data_path, 'app_events.csv'), usecols=('is_installed', 'event_id', 'app_id'),
                         dtype={'is_installed': np.int8, 'event_id': np.int64, 'app_id': np.int64})

In [4]:
all_apps = app_events.app_id.unique()

In [5]:
res = sp.csr_matrix((len(app_events.event_id.unique()), len(app_events.app_id.unique())))
events = []

In [6]:
current_event_id = None
item = None

counter = 0
v = None

for row in app_events.values:
    event_id, app_id = row[0], row[1]
    if current_event_id is None:
        current_event_id = event_id
        item = dict(zip(all_apps, np.zeros(len(all_apps))))
    if current_event_id == event_id:
        item[app_id] = 1
    else:
        v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
        res = sp.hstack([res, v])
        
        current_event_id = event_id
        item = dict(zip(all_apps, np.zeros(len(all_apps))))
        events.append(current_event_id)
    counter += 1
    
    if len(events) % 50000 == 0:
        print counter

if item is not None:
    v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
    res = sp.hstack([res, v])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


KeyboardInterrupt: 

In [ ]:
joblib.dump(res, 'sparse_app_is_installed')
joblib.dump(events, 'events')